<a href="https://colab.research.google.com/github/im-nandha/LLM-Powered-Booking-Analytics-QA-System/blob/main/llm_based_question_answering_fast_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install fastapi uvicorn transformers sentence-transformers faiss-cpu nest_asyncio pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [4]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import faiss
import numpy as np
from transformers import pipeline
from sentence_transformers import SentenceTransformer

# Apply nest_asyncio to run FastAPI in Jupyter Notebook
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Load dataset
df = pd.read_csv("hotel_bookings1.csv")

# Convert relevant columns to a single text format for retrieval
df["combined_text"] = df.apply(lambda row:
    f"Hotel: {row['hotel']}, Canceled: {row['is_canceled']}, Lead Time: {row['lead_time']} days, "
    f"Arrival: {row['arrival_date_month']} {row['arrival_date_year']}, Price: ${row['adr']}, "
    f"Country: {row['country']}", axis=1)

# Load Sentence Transformer model for text embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embeddings for dataset
embeddings = embedding_model.encode(df["combined_text"].tolist(), convert_to_tensor=False)

# Store embeddings in FAISS for retrieval
dimension = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(embeddings))

# Load LLM model for answering questions
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Define request model
class QueryRequest(BaseModel):
    question: str

@app.get("/")
def home():
    return {"message": "Hotel Booking QA API is running!"}

@app.post("/ask")
def ask_question(query: QueryRequest):
    # Convert question to embedding
    query_embedding = embedding_model.encode([query.question])

    # Find the most relevant document
    _, closest_doc_idx = faiss_index.search(np.array(query_embedding), k=1)

    # Retrieve the best-matching text from the dataset
    best_match = df.iloc[closest_doc_idx[0][0]]["combined_text"]

    # Use retrieved text as context for LLM
    response = qa_pipeline(question=query.question, context=best_match)

    return {"question": query.question, "answer": response["answer"], "context": best_match}

# Run FastAPI in Jupyter Notebook
uvicorn.run(app, host="127.0.0.1", port=8000)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
INFO:     Started server process [303]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [303]
